Get only selected new files. fsspec doesn't allow pattern matching and issues downloading to S3 makes it less than intuitive.

In [ ]:
!pip install paramiko -q

In [ ]:
import os 
import fsspec 
import gzip 
import shutil 

In [ ]:
fs = fsspec.filesystem('sftp', host="fuoco.geog.umd.edu", username="fire", password="burnt")

# we want the collection 2 data from SNPP
snpp = fs.ls('/data/VIIRS/C2/VNP14IMGML/')
print(f"{len(snpp)} monthly files availabe.\n{snpp[0]} to {snpp[-1]}")

snpp

In [ ]:
# in this case, I just want the last 6 months 
# grab filenames from above output

files = ['/data/VIIRS/C2/VNP14IMGML/VNP14IMGML.202507.C2.04.csv.gz',
 '/data/VIIRS/C2/VNP14IMGML/VNP14IMGML.202508.C2.04.csv.gz',
 '/data/VIIRS/C2/VNP14IMGML/VNP14IMGML.202509.C2.04.csv.gz',
 '/data/VIIRS/C2/VNP14IMGML/VNP14IMGML.202510.C2.04.csv.gz',
 '/data/VIIRS/C2/VNP14IMGML/VNP14IMGML.202511.C2.04.csv.gz',
 '/data/VIIRS/C2/VNP14IMGML/VNP14IMGML.202512.C2.04.csv.gz']

In [ ]:
# download here as an intermediate step
# downloading direct to public bucket is buggy
for f in files:
    print(f) 
    fs.get(f, ".")

In [ ]:
input_dir = '/projects/fire-utils/' 
output_dir = '/projects/my-public-bucket/FEDSstaging/FEDSinput/VIIRS/VNP14IMGML/'

os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith('.csv.gz'):
        input_path = os.path.join(input_dir, filename)
        output_filename = filename[:-3]  # remove the '.gz'
        output_path = os.path.join(output_dir, output_filename)

        with gzip.open(input_path, 'rt') as f_in, open(output_path, 'wt') as f_out:
            shutil.copyfileobj(f_in, f_out)

        print(f"Extracted {filename} to {output_path}")